## Подготовка среды

В этой части кода мы подгружаем необходимые библиотеки, подключаемся к vk API, задаем функции, которые будут использовать в процессе сбора данных. 

Описание используемых функций:

`create_dates`: функция, возвращающая два спика дат в формате `datetime`. Первый список содержит даты, с которых в коде в дальнейшем будет происходить начало поиска постов, второй список - дату, до которой будет происходить поиск постов. Разница между датой начала и конца стоит в 1 день, чего достаточно для сбора постов по ключевым словам. VK API ставит ограничения на запрос постов в 1000 штук, из-за чего для поиска постов в определенном периоде мы будем проходиться по дням. 

`divide_dates`: функция, которая делит списки дат на списки меньшей длины. Такая уловка нужна для того, чтобы API не блокировало доступ.

**И еще пару моментов**

1. В функциях, собирающих данные, стоят достаточно большие значения `sleep()`. При меньших значениях vk API блокирует доступ на n-ой итерации. Лучше не делать эти значения меньше - с оставленными в ноутбуке значениями все точно сработало.

2. Также в функциях я достаточно часто вывожу количество собранных данных. Выглядит это все не самым удобным образом: вывод под вызовом функции очень нагроможден. Однако у vk API есть одна забавная особенность: когда с нашего токена больше нельзя делать запрос по какой-то причине (слишком часто просим, например), то API не выдает нам ошибку, а просто возвращает пустой список. Таким образом можно какую-то часть цикла просто не собирать данные, и мы об этом даже и не узнаем, а частый вывод помогает это контролировать. 

In [11]:
import pandas as pd
import numpy as np
import vk
from time import sleep
import json
from tqdm.notebook import tqdm
import orjsonl

with open('keywords.txt', 'r', encoding='utf-8') as f:
    keywords = f.read().split(', ')

In [19]:
token = ''
api = vk.API(access_token=token) 

In [14]:
def create_dates(start_date: tuple, end_date: tuple, shift: int, found_dates=None):
    
    '''
    start_date: начало поиска
    end_date: конец поиска
    shift: сколько дней пропускаем из всего периода (shift = 2, то есть, берем каждый второй день)
    found_dates: параметр на случай, если хотим дополнительно проверить то, чтобы уже использованные даты не 
    входили в новые списки дат
    '''
    
    start_date = datetime.datetime(*start_date)
    end_date = datetime.datetime(*end_date)

    start_dates = []
    end_dates = []

    for n in range((end_date - start_date).days // shift + 1):
        current_date = start_date + datetime.timedelta(days=shift * n)
        start_dates.append(current_date)
        end_dates.append((current_date + datetime.timedelta(days=1)))

    if found_dates:
        start_dates = [i for i in start_dates if i not in found_dates[0]]
        end_dates = [i for i in end_dates if i not in found_dates[1]]
    
    assert len(start_dates) == len(end_dates)
    
    return (start_dates, end_dates)

In [15]:
def divide_dates(start_dates:list, end_dates:list, len_list: int):
    '''
    start_dates: список дат начала поиска, полученный из функции create_dates
    end_dates: аналогично
    len_list: длина новых списков
    '''
    
    all_dates = []
    n = len(start_dates)
    
    start = 0
    end = len_list
    while True:
        if start == (n // len_list) * len_list:
            all_dates.append([start_dates[start:], end_dates[start:]])
            break
    
        all_dates.append([start_dates[start:end], end_dates[start:end]])
        start += len_list
        end += len_list
        
    return all_dates

# Сбор постов по ключевым словам

In [ ]:
def get_posts(keywords: list, start_dates: list, end_dates: list, file: str):
    l = 0
    cols = ['comments', 'marked_as_ads', 'date', 'from_id', 'id', 'likes', 'owner_id', 'reposts', 'text']
    # идем по каждому ключевому слову
    for kw in tqdm(keywords):
    
        sleep(2)
        
        # идем по каждой дате
        for sd, ed in tqdm(zip(start_dates, end_dates)):
        
            sleep(3)
            
            # переводим дату в нужный формат
            sd = sd.timestamp()
            ed = ed.timestamp()
            
            all_posts = []
            offset = 0
        
            while True:
                
                sleep(5)
                
                try:
                    
                    # запрашиваем посты
                    posts = api.newsfeed.search(q=kw, extended=1, count=200, start_time=sd, end_time=ed, 
                                                v=5.81, offset=offset)['items']
                    if len(posts) == 0:
                        print('oops')
                
                    for post in posts:
                        
                        # не у всех постов есть ключ comments: поэтому здесь тоже пользуемся try except
                        try:
                            
                            # сохраняем только те посты, у которых есть комментарии. делаем это в процессе сбора, 
                            # так как посты занимают много места
                            
                            if post['comments']['count'] == 0:  
                                continue
                                
                        except Exception:
                            sleep(1)
                            continue

                        res = {key: post[key] for key in cols}
                        res['comments'] = res['comments']['count']
                        res['likes'] = res['likes']['count']
                        res['reposts'] = res['reposts']['count']
                        try:
                            res['views'] = res['views']['count']
                        except Exception:
                            res['views'] = -99

                        orjsonl.append(file, res)
                
                    # если больше постов по данному ключевому посту нет, то останавливаем цикл
                    if len(posts) < 200:
                        break
                        
                    # даже с установленным offset vk api не позволяет собирать больше 1000 постов, поэтому
                    # на 5 итерации можем останавливать
                    if offset == 800:
                        break
                        
                    offset += 200
                
                except Exception as e:
            
                    print(e)

                    sleep(5)
            
            
        if l % 50 == 0:
            print(f'posts found: {len(orjsonl.load(file))}')

In [ ]:
# пример запуска функции

start_dates, end_dates = create_dates((2023, 8, 26), (2023, 10, 25), 1)
dates = divide_dates(start_dates, end_dates, 13)

posts_file = 'posts8_10.json'
keywords = ['женщина', 'феминизм', 'гендер', 'женский', 'девушка']

for i in tqdm(dates):
    
    start_dates = i[0]
    end_dates = i[1]
    
    get_posts(keywords, start_dates, end_dates, posts_file)

In [16]:
def comments_from_posts(posts: list, start_at: 0, file: str, search: False):

    '''
    posts: список постов
    start_at: с какого поста начинаем собирать комментарии: нужно на случай, если код случайно упадет
    file: название файла
    search: True/False, если True - собираем комментарии только с ключевыми словами. Пока что нам это не надо, 
    мы хотим собрать все комментарии, так что ставим False
    '''
    i = start_at
    filtered_comments = []
    cols = ['id', 'from_id', 'date', 'text']
    
    # идем по постам
    for post in tqdm(posts[start_at:]): 
        
        sleep(0.5)
        
        try:
            
            comments = api.wall.getComments(owner_id = post['owner_id'], post_id=post['id'], v=5.81)['items']
            
            # когда vk api ломается, оно не выдает ошибку: оно просто выдает пустой список, так что делаем
            # проверку. 0 комментариев у нас не может быть: когда мы собирали посты, мы собирали только те, у
            # которых есть комментарии
            
            
            
            for comm in comments:
            
                if comm['text'] == '':
                    continue
    
                        

                res = {key: comm[key] for key in cols}
                res['owner_id'] = post['owner_id']
                res['post_id'] = post['id']
                filtered_comments.append(res)

                      
            i += 1
            if i % 1000 == 0: 
                print(i)
                # сохраняем собранные комментарии в файл
                orjsonl.extend(file, filtered_comments)
                print(f'post: {i}, found comments: {len(filtered_comments)}')
                filtered_comments = []
                
                    
        except Exception as e:
                print(e)
                sleep(5)
                continue
                
    print(filtered_comments)
    orjsonl.extend(file, filtered_comments)

In [21]:
posts = orjsonl.load('posts_pol2_2_3_filtered.json')

In [25]:
comments_file = 'comm_pol2_2_3.json'
comments_from_posts(posts, 0, comments_file, False)

  0%|          | 0/7243 [00:00<?, ?it/s]

212. Access to post comments denied. request_params = {'method': 'wall.getComments', 'oauth': '1', 'owner_id': '195548187', 'post_id': '10597', 'v': '5.81'}
1000
post: 1000, found comments: 4806
2000
post: 2000, found comments: 4705
212. Access to post comments denied. request_params = {'method': 'wall.getComments', 'oauth': '1', 'owner_id': '84785947', 'post_id': '23479', 'v': '5.81'}
212. Access to post comments denied. request_params = {'method': 'wall.getComments', 'oauth': '1', 'owner_id': '405755646', 'post_id': '67965', 'v': '5.81'}
15. Access denied: post was deleted. request_params = {'method': 'wall.getComments', 'oauth': '1', 'owner_id': '-82659298', 'post_id': '1055275', 'v': '5.81'}
3000
post: 3000, found comments: 4708
4000
post: 4000, found comments: 4731
212. Access to post comments denied. request_params = {'method': 'wall.getComments', 'oauth': '1', 'owner_id': '262288216', 'post_id': '875', 'v': '5.81'}
5000
post: 5000, found comments: 4678
212. Access to post commen

In [11]:
c = pd.DataFrame(orjsonl.load('comm_pol_test4.json'))

In [13]:
pd.DataFrame(posts)

,comments,marked_as_ads,date,from_id,id,likes,owner_id,reposts,text,views
0,4,0,1696188243,-91382211,9028,86,-91382211,10,Церковные даты октября. \n\n1. Воскресенье. Ик...,-99
1,5,0,1696185000,-1180950,299870,276,-1180950,9,Алексей Дикий\n25 апреля 1889 - 1 октября 1955...,-99
2,59,0,1696182693,-211255544,35624,1115,-211255544,279,Анонс. Полезная авторская рубрика.\n\nЦерковны...,-99
3,1,0,1696180800,-72671313,2243658,53,-72671313,5,Фестиваль против неонацизма и неофашизма прохо...,-99
4,110,0,1696179601,-201656522,99505,11,-201656522,5,#околополит_анкета@politicumforum \n\n[id58504...,-99
5,1,0,1696179600,-56331540,125576,22,-56331540,7,"✨ О ПОИСКАХ ДУШИ, ИЗМЕРЕНИЯХ, ПЕРВОПРЕДКЕ И НЕ...",-99
6,3,0,1696177242,-170746489,1277,69,-170746489,3,"Дорогие братья и сестры, с праздником! Праздни...",-99
7,2,0,1696177200,-81668509,227218,10,-81668509,1,ПОД ПЕНЬЕ ТРУБ И РОКОТ БАРАБАНОВ\n\n#МИР_ВОКРУ...,-99
8,1,0,1696174563,-219483487,59780,0,-219483487,0,В Испании проходит фестиваль против неонацизма...,-99
9,25,0,1696174200,-26493942,7250247,98,-26493942,6,Фестиваль против неонацизма и неофашизма прохо...,-99
